### Importing all the required libraries

In [62]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer,WordNetLemmatizer
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from autocorrect import Speller
import emoji

In [2]:
data1 = pd.read_json(r"C:\Users\srava\Downloads\hotel_reviews (2).json")

In [3]:
data1

,review,sentiment
0,The walls were thin and could hear noise from ...,positive
1,The walls were thin and could hear noise from ...,negative
2,The food was overpriced and not very tasty. 😐,neutral
3,"An average stay, nothing special but no major ...",positive
4,Loved the spa and pool area! A truly relaxing ...,negative
...,...,...
3995,"An average stay, nothing special but no major ...",neutral
3996,Very clean rooms and excellent location. Highl...,positive
3997,"The hotel was okay, not great but not terrible...",negative
3998,Unfriendly staff and the breakfast was disappo...,neutral


### Converting json file to csv and saved it

In [8]:
data2 = data1.to_csv(r'C:\\Users\srava\Downloads.csv')

In [26]:
data_c = pd.read_csv(r"C:\Users\srava\Downloads\hotel_reviews (2).csv")

In [27]:
data_c.head()

,Unnamed: 0,review,sentiment
0,0,The walls were thin and could hear noise from ...,positive
1,1,The walls were thin and could hear noise from ...,negative
2,2,The food was overpriced and not very tasty. 😐,neutral
3,3,"An average stay, nothing special but no major ...",positive
4,4,Loved the spa and pool area! A truly relaxing ...,negative


In [28]:
data_2 = data_c.copy()

In [29]:
data_2 = data_2.drop('Unnamed: 0',axis=1)

In [30]:
data_2

,review,sentiment
0,The walls were thin and could hear noise from ...,positive
1,The walls were thin and could hear noise from ...,negative
2,The food was overpriced and not very tasty. 😐,neutral
3,"An average stay, nothing special but no major ...",positive
4,Loved the spa and pool area! A truly relaxing ...,negative
...,...,...
3995,"An average stay, nothing special but no major ...",neutral
3996,Very clean rooms and excellent location. Highl...,positive
3997,"The hotel was okay, not great but not terrible...",negative
3998,Unfriendly staff and the breakfast was disappo...,neutral


In [59]:
feature_var= data_2[['review']]

# Step:1 Converting text to lower case

In [31]:
" ".join(data_2['review']).islower()

False

In [126]:
def eda(data,name):
    html_=data[name].apply(lambda x:True if re.search("<.+?>",x) else False).sum()
    if html>0:
        print("html tags are there")

In [60]:
feature_var

,review
0,The walls were thin and could hear noise from ...
1,The walls were thin and could hear noise from ...
2,The food was overpriced and not very tasty. 😐
3,"An average stay, nothing special but no major ..."
4,Loved the spa and pool area! A truly relaxing ...
...,...
3995,"An average stay, nothing special but no major ..."
3996,Very clean rooms and excellent location. Highl...
3997,"The hotel was okay, not great but not terrible..."
3998,Unfriendly staff and the breakfast was disappo...


## Finding the html tags, url's, Special characters in the dataset

In [52]:
feature_var.apply(lambda x:True if re.search("<.+?>",x) else False).sum()
# there are no html tags in the dataset

0

In [53]:
feature_var.apply(lambda x:True if re.search("http[s]?://.+? +",x) else False).sum()
# there are no url's in the dataset

0

In [54]:
feature_var.apply(lambda x:True if re.search("[]()*\-.,@#$%^&0-9]",x) else False).sum()
# there are special characters in the dataset

4000

In [78]:
def convert_emojis_to_text(text):
    return emoji.demojize(text)
feature_var['review']=feature_var['review'].apply(convert_emojis_to_text)

C:\Users\srava\AppData\Local\Temp\ipykernel_30464\726179679.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_var['review']=feature_var['review'].apply(convert_emojis_to_text)


In [94]:
def txtpp(y,correct,emojii):
    from textblob import TextBlob
    spell=Speller(lang="en")
    y=y.lower()
    y=re.sub("[]:()*\-.,@#$%^&0-9]"," ",y)
    y=re.sub("\s+"," ",y)
    if correct=="t":
        y=TextBlob(y).correct().string
    else:
        y=spell(y)
    return y

In [95]:
feature_var['review']=feature_var["review"].apply(txtpp,args=("spell",False))

C:\Users\srava\AppData\Local\Temp\ipykernel_30464\738733230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_var['review']=feature_var["review"].apply(txtpp,args=("spell",False))


In [96]:
feature_var['review']

0       the walls were thin and could hear noise from ...
1       the walls were thin and could hear noise from ...
2       the food was overpriced and not very tasty neu...
3       an average stay nothing special but no major p...
4       loved the spa and pool area! a truly relaxing ...
                              ...                        
3995    an average stay nothing special but no major p...
3996    very clean rooms and excellent location highly...
3997    the hotel was okay not great but not terrible ...
3998    unfriendly staff and the breakfast was disappo...
3999    very clean rooms and excellent location highly...
Name: review, Length: 4000, dtype: object

In [93]:
y = 'the food was overpriced and not very tasty   n'
re.sub("[]:()*\-.,@#$%^&0-9]"," ",y)
re.sub("\s+"," ",y)

'the food was overpriced and not very tasty n'

In [97]:
feature_var['review'].apply(lambda x:True if re.search("\s+",y) else False).sum()

4000

In [100]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\srava\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [106]:
stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [103]:
ps = PorterStemmer()
ls = LancasterStemmer()
ss = SnowballStemmer(language='english')

In [107]:
def advpp(x,stemm):
    l=[]
    for word in word_tokenize(x):
        if word in stop:
            pass
        else:
            if stemm == 'p':
                l.append(ps.stem(word))
            elif stemm == 'l':
                l.append(ps.stem(word))
            elif stemm == 's':
                l.append(ps.stem(word))
            else:
                l.append(word)
    return " ".join(l)

In [113]:
feature_var['review']=feature_var['review'].apply(advpp,args='p')

In [116]:
def txtpp(y):#,correct,emojii):
    #from textblob import TextBlob
    #spell=Speller(lang="en")
    #y=y.lower()
    y=re.sub("[^a-z]"," ",y)
    y=re.sub("\s+"," ",y)
    #if correct=="t":
       # y=TextBlob(y).correct().string
   # else:
        #y=spell(y)
    return y

In [117]:
feature_var['review']=  feature_var['review'].apply(advpp)

In [119]:
feature_var['review']

0       wall thin could hear nois room good stay smili...
1       wall thin could hear nois room good stay disap...
2                           food overpr tasti neutral fac
3       averag stay noth special major problem smiling...
4       love spa pool area truli relax experi disappoi...
                              ...                        
3995    averag stay noth special major problem neutral...
3996    clean room excel locat highli recommend smilin...
3997     hotel okay great terribl either disappointed fac
3998    unfriendli staff breakfast disappoint neutral fac
3999    clean room excel locat highli recommend smilin...
Name: review, Length: 4000, dtype: object

In [120]:
cv=CountVectorizer(ngram_range=(1,1))

In [122]:
final_cv=cv.fit_transform(feature_var['review']).toarray()

In [124]:
final_cv

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]], dtype=int64)